# 트랜스포머 아키텍처
: 셀프 어텐션을 통해 문장의 맥락을 포착하는데 사용되는 기법(문장 속 모든 단어의 관련성과 맥락을 학습)

### * 셀프 어텐션(Self-Attention)
: 시퀸스의 각 단어가 다른 단어들과 어떻게 관련되어 있는지 계산하는 메커니즘
1. 쿼리, 키, 값의 벡터를 생성(세 개의 벡터)
2. 어텐션 스코어 : 각 단어의 쿼리 벡터와 모든 단어의 키 벡터를 곱한다.
3. 소프트맥스 : 어텐션 스코어를 소프트맥스 함수에 통과시켜 가중치를 만든다.
4. 어텐션 값 : 가중치를 값 벡터에 곱하여 가중합을 계산한다.
    - 공식 : Attebtui(Q, K, V) = softmax(QK^T/루트dk)V 이런 느낌...?

### * 인코더-디코더 구조
- 인코더
1. 입력 임베딩 : 입력된 각 단어를 특정 차원의 벡터로 변환한다.
2. 포지셔널 인코딩 : 단어의 위치 정보를 추가하여 순서의 의미를 반영한다.
3. 인코더 레이어 : 각 레이어는 셀프 어텐션과 피드포워드 네트워크로 이루어져 있다. ⭐️

- 디코더
1. 출력 임베딩 : 생성된 각 단어를 특정 차원의 벡터로 변환한다.
2. 포지셔널 인코딩 : 위치정보 추가
3. 디코더 레이어 : 각 레이어는 셀프 어텐션과 피드포워드 네트워크, 인코더-디코더 어텐션으로 이루어져 있다.⭐️

In [3]:
# 기존에 사용했던 데이터 들고오기
import os, pathlib, shutil, random
from tensorflow import keras

batch_size = 32
base_dir = pathlib.Path('aclImdb')
val_dir = base_dir / 'va'
train_dir = base_dir / 'train'

# for category in ('neg', 'pos'):
#     os.makedirs(val_dir / category)
#     files = od.listdir(train_dir / category)
#     random.Random(1337).shuffle(files)

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size = batch_size)

val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size = batch_size)

test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size = batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [5]:
text_only_train_ds = train_ds.map(lambda x, y : x)

In [8]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
    max_tokens = max_tokens,
    output_mode = 'int',
    output_sequence_length = max_length,
)

text_vectorization.adapt(text_only_train_ds)

In [9]:
int_train_ds = train_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls = 4)

int_val_ds = val_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls = 4)

int_test_ds = test_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls = 4)

# 멀티 헤드 어텐션 
: 셀프 어텐션의 여러 헤드를 병렬로 사용하는 방식(각각 헤드가 다른 부분을 처리하여 다양한 표현을 학습할 수 있다.)

- 다중 헤드 분할 : 입력을 여러 개의 서브스페이스로 분할한다.
- 개발 어텐션 계산 : 각 서브스페이스에서 셀프 어텐션을 수행한다.
- 병합 : 각 헤드의 출력을 병합하여 다음 레이어로 전달한다.

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer): # layer을 상속받아 Layer 처리 진행
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs): # 각 차원 수를 전달받아 처리한다.
        super(TransformerEncoder, self).__init__(**kwargs) # 다중입력처리를 입력받아 부모에게 전달
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim) # multi
        self.dense_proj = keras.Sequential( # 두 개의 dense 층으로 이루어진 모델 = 피드포워드 네트워크(신경망)
        # 피드포워드 네트워크 : 정규화된 출력을 두 개의 밀집 레이어에 통과시킨다.
            [layers.Dense(dense_dim, activation='relu'),
             layers.Dense(embed_dim)]
        )
        self.layernorm_1 = layers.LayerNormalization() # 정규화 층
        self.layernorm_2 = layers.LayerNormalization() # 정규화 층

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :] # 차원 확장
        attention_output = self.attention(inputs, inputs, attention_mask=mask) # 멀티헤드 어텐션 레이어의 출력(쿼리, 키, 값)
        proj_input = self.layernorm_1(inputs + attention_output) # 입력과 어텐션을 더한 값(정규화 처리)
        proj_output = self.dense_proj(proj_input) # 통과
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self): # 구성요소 저장 함수
        config = super().get_config()
        config.update({
        'embed_dim' : self.embed_dim,
        'num_heads' : self.num_heads,
        'dense_dim' : self.dense_dim,
        })
        return config

# 트랜스포머 인코더 순서 : 멀티 헤드 셀프 어텐션 -> 잔차 연결 -> 정규화 -> 피드포워드 신경망 -> 잔차 연결 -> 정규화

In [16]:
# 텍스트 분류 작업 진행
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype='int64')
x = layers.Embedding(vocab_size, embed_dim)(inputs) # 임베딩 벡터 변환
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x) # 트랜스포머 인코더
x = layers.GlobalMaxPooling1D()(x) # 시퀀스 차원에서 최대풀링을 수행하여 고정 크기의 특징 벡터를 얻는다.
x = layers.Dropout(0.5)(x)
# 위 코드에서 써놓은 트랜스포머 인코더 순서를 모두 마친 후 x에 값 저장

outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 transformer_encoder_1 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 257 

In [19]:
model = keras.models.load_model('transformer_encoder.h5', custom_objects={'TransformerEncoder' : TransformerEncoder})
print(f'테스트 정확도 : {model.evaluate(int_test_ds)[1]:.3f}')

293/782 [==========>...................] - ETA: 23:54 - loss: 1.2959 - accuracy: 0.5076


KeyboardInterrupt



# 위치 인코딩(Position Encoding)
: 각 토큰의 위치 정보를 인코딩하여 입력에 추가하는 것(모델이 입력 시퀀스의 순서 정보를 학습)
- 트랜스포머 모델은 순차적인 정보 처리를 위한 재귀적 구조가 존재하지 않는다.

1. 고정 위치 인코딩(Fixed Position Encoding)
: 계산이 간단하고 구현하기 쉽지만 입력 시퀀스의 길이가 고정되어야 한다.
- sin()과 cos() 함수를 사용하여 각 토큰의 위치정보를 인코딩한다.
- PE(pos, 2i) = sin(pos/10000^(2i/d_model))
- PE(pos, 2i+1) = cos(pos/10000^(2i/d_model))

2. 학습 가능한 위치 인코딩(Learned Position Encoding)
: 입력 시퀀스의 길이는 제한이 없지만 별도의 학습과정이 필요하다.
- 위치 정보를 학습 가능한 매개변수로 표현한다.

In [25]:
# 위치 임베딩(고정 위치 인코딩)
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer): # 입력 시퀀스 길이가 고정되어야 하므로,
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs): # 입력 시퀀스의 최대 길이, 입력 토큰의 차원 수, 출력 임베딩 차원 수
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim) # 입력 토큰 임베딩
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim) # 위치 정보 임베딩
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1] # 입력 시퀀스 길이
        positions = tf.range(start=0, limit=length, delta=1) # 위치 정보를 나타내는 텐서를 생성
        embedded_positions = self.position_embeddings(positions)
        embedded_tokens = self.token_embeddings(inputs)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
        
    def get_config(self):
        config = super().get_config()
        config.update({
            'output_dim': self.output_dim,
            'sequence_length': self.sequence_length,
            'input_dim': self.input_dim,
        })
        return config

In [27]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype='int64')
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding_1 (Po  (None, None, 256)         5273600   
 sitionalEmbedding)                                              
                                                                 
 transformer_encoder_3 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_3 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                           

In [32]:
model = keras.models.load_model('full_transformer_encoder.h5',
                                custom_objects={'TransformerEncoder':TransformerEncoder,
                                               'PositionalEmbedding' : PositionalEmbedding})

print(f'테스트 정확도 : {model.evaluate(int_test_ds)[1]:.3f}')

782/782 [==============================] - 2095s 3s/step - loss: 1.3416 - accuracy: 0.5244
테스트 정확도 : 0.524


## BoW 모델 대신 언제 시퀸스 모델을 사용할까?
=> 단어의 순서를 무시하고 단어의 빈도만 처리하는 것 대신 단어의 순서를 고려해야 할 때가 언제인지

1. 문맥을 고려해야 할 때 : dog bites man, man bites dog
2. 언어 모델링 : 다음 단어를 예측하거나 텍스트를 생성할 때
3. 긴 의존성 : RNN과 트랜스포머는 긴 시퀸스 내에서 멀리 떨어진 단어 간의 의존성 파악할 수 있다.